This script will take two fires from mtbs and try to use mighty mosaic on them to compare to EE, in hopes it will remove the tiling effect.  MightyMosaic docs: https://github.com/AurelienColin/MightyMosaic#introduction

Note we don't have any fires in the Forest Tundra Ecoregion... that will need to be fixed. 

In [1]:
import pandas as pd
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["SM_FRAMEWORK"] = "tf.keras"
import tensorflow
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.python.lib.io import file_io
from tensorflow.python.keras.optimizer_v2.adam import Adam
import os
import segmentation_models as sm
import matplotlib.pyplot as plt
import numpy as np
#from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.layers import concatenate, Conv2DTranspose, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, Input, AvgPool2D
from tensorflow.keras.models import Model
from keras_unet_collection import models
import tensorflow_addons as tfa
import geopandas as gpd
import logging
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

Segmentation Models: using `tf.keras` framework.


/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a 

In [2]:
min_max = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_glob_norm.csv").reset_index(drop = True)

#6, 7, 8 correspond to dNBR, dNDVI, dNDII, if you change these bands this must change. .
min_max = min_max[['7', '8', '9']]

class img_gen_vi(tensorflow.keras.utils.Sequence):

    """Helper to iterate over the data (as Numpy arrays).
    Inputs are batch size, the image size, the input paths (x) and target paths (y)
    """

    #will need pre defined variables batch_size, img_size, input_img_paths and target_img_paths
    def __init__(self, batch_size, img_size, input_img_paths):
	    self.batch_size = batch_size
	    self.img_size = img_size
	    self.input_img_paths = input_img_paths
	    self.target_img_paths = input_img_paths

    #number of batches the generator is supposed to produceis the length of the paths divided by the batch siize
    def __len__(self):
	    return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_img_paths = self.input_img_paths[i : i + self.batch_size] #for a given index get the input batch pathways (x)
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size] #for a given index get the input batch pathways (y)
		
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32") #create matrix of zeros which will have the dimension height, wideth, n_bands), 8 is the n_bands
        
  
         #start populating x by enumerating over the input img paths
        for j, path in enumerate(batch_img_paths):

            #load image
            img =  np.round(np.load(path), 3)[:, :, 7:10]

            # img = img * 1000
            img = img.astype(float)
            img = np.round(img, 3)
            img[img == 0] = -999

            img[np.isnan(img)] = -999


            img[img == -999] = np.nan

            in_shape = img.shape
            
            #turn to dataframe to normalize
            img = img.reshape(img.shape[0] * img.shape[1], img.shape[2])
			
            img = pd.DataFrame(img)
			
            img.columns = min_max.columns
			
            img = pd.concat([min_max, img]).reset_index(drop = True)


            #normalize 0 to 1
            img = pd.DataFrame(scaler.fit_transform(img))
			
            img = img.iloc[2:]
#
#             img = img.values.reshape(in_shape)
            img = img.values.reshape(in_shape)

#             replace nan with -1
            img[np.isnan(img)] = -1

#apply standardization
# img = normalize(img, axis=(0,1))

            img = np.round(img, 3)
            #populate x
            x[j] = img#[:, :, 4:] index number is not included, 


        #do tthe same thing for y
        y = np.zeros((self.batch_size,) + self.img_size, dtype="uint8")

        for j, path in enumerate(batch_target_img_paths):

            #load image
            img =  np.round(np.load(path), 3)[:, :, -1]

            img = img.astype(int)

            img[img < 0] = 0
            img[img >1] = 0
            img[~np.isin(img, [0,1])] = 0

            img[np.isnan(img)] = 0
            img = img.astype(int)

            y[j] = img
  
       

        return x, y



In [3]:
#batch size and img size
BATCH_SIZE = 45
GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

#image size
img_size = (16, 16)

#number of classes to predict
num_classes = 1

#nbac mtbs model
model_1 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/models/nbac_mtbs_modis_16_global_norm.tf", 
                                           custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
                                                           'recall':sm.metrics.Recall(threshold = 0.5),
                                                            'f1-score': sm.metrics.FScore(threshold=0.5),
                                                             'iou_score': sm.metrics.IOUScore(threshold=0.5),
                                                              'accuracy': 'accuracy'})

#nbac mtbs model with 85% dnbr threshold
# model_2 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/models/nbac_mtbs_regularize_50_global_norm_85.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})

# model_3 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/combined_good_old_dnbr.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})
# #combined 85
# model_4 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/combined_good_85.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})

# #russia onl
# model_5 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/russia_good_no_regularize.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})
# #russia old dnbr method
# model_6 = tensorflow.keras.models.load_model("/explore/nobackup/people/spotter5/cnn_mapping/Russia/models/russia_good_no_regularize_old_dnbr.tf", 
#                                            custom_objects={'precision':sm.metrics.Precision(threshold=0.5), 
#                                                            'recall':sm.metrics.Recall(threshold = 0.5),
#                                                             'f1-score': sm.metrics.FScore(threshold=0.5),
#                                                              'iou_score': sm.metrics.IOUScore(threshold=0.5),
#                                                               'accuracy': 'accuracy'})



Number of devices: 1


In [5]:
def predict_model(model, generator, name, fid, count):
    
    '''
    model: tensorflow model to predict
    generator: keras generator with the images to predict on
    name: string, model name\
    fid: variable I was looping through
    count: count retained earlier
    '''
    #get the results from the nbac and mtbs model
    model_1_res = model.evaluate_generator(generator, 100)

    iou = np.round(model_1_res[-2], 2)
    precision = np.round(model_1_res[-5], 2)
    recall = np.round(model_1_res[-4], 2)
    f1 = np.round(model_1_res[-3], 2)
    accuracy = np.round(model_1_res[-1], 2)

    #make new dataframe with scores
    in_df = pd.DataFrame({
        'Model': [name],
        'FID': [fid],
        'Count': [count],
        'IOU': [iou],
        'Precision': [precision],
        'Recall': [recall],
        'F-1': [f1],
        'Accuracy': [accuracy]
                        }, index=[0])  # Explicitly setting index to [0] for a single row

    return in_df

First lets do just the grid so I need to loop through the anna good polygon ids in each grid cell, get the IOU for that and save it.
I will combine these into one pandas dataframe and then output the file as a shapefile so we can view it spatially. I will also do this two ways with the old dnbr and with the new dnbr in the testing set. 

In [61]:
#first these are all the good anna polygon ids, use this for joining later
# good_ids = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/good_polys_anna.shp')

#for the grids I have two ids, FID which is the fishnet grid cells to loop through, and ID which is teh good anna polygon nids
fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/grid.shp')
fish_good['Grid_ID'] = fish_good['Grid_ID'].astype(int)
#all the fishnet ids to loop through
all_fid = fish_good['Grid_ID'].unique().tolist()

#get all the testing full pathways to predict on, will need to filter fish good with this
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv')['ID'].tolist()


#now I need to get the chunked files which match the fire ids to make new training, validation and testing times
#path to the chunked files
chunked_85 =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128')
chunked_old =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128')

def filter_chunked_85(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

def filter_chunked2(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    # filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

def filter_chunked_old(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked


#new dnbr composite method
testing_names_85 = filter_chunked_85(testing_names, chunked_85)
testing_names_old = filter_chunked_old(testing_names, chunked_old)



#save all dataframes
final = []

for fid in all_fid:
    
    # print(f"Processing {fid}")
    
    #sub shapefile for the grid
    sub_grid = fish_good[fish_good['Grid_ID'] == fid]
                         
    #get the anna fire ids in this fid
    anna_in_fid = sub_grid['Fire_ID'].unique().tolist()
                        
    count = len(anna_in_fid)
    
    #get full pathway to the anna ids in the fids for new dnbr method
    model_test_85 = filter_chunked2(anna_in_fid, testing_names_85)
    model_test_old = filter_chunked2(anna_in_fid, testing_names_old)
    
    #get the batch sie
    if len(model_test_85) <= 45:
        
        batch_size_85 = len(model_test_85)
       
    else:
        batch_size_85 = 45
        
    if len(model_test_old) <= 45:
        
        batch_size_old = len(model_test_old)
       
    else:
        batch_size_old = 45

        
    if len(model_test_85) > 0 and (len(model_test_old)) > 0:
    
        #create an image generator for this fid and then predict
        models_vi_gen_85 =  img_gen_vi(batch_size_85, img_size, model_test_85)
        models_vi_gen_old =  img_gen_vi(batch_size_old, img_size, model_test_old)

        mtbs_nbac = predict_model(model_1, models_vi_gen_old, 'MTBS_NBAC', fid, count)
        mtbs_nbac_85 = predict_model(model_2, models_vi_gen_85, 'MTBS_NBAC_85', fid, count)
        combined = predict_model(model_3, models_vi_gen_old, 'Combined', fid, count)
        combined_85 = predict_model(model_4, models_vi_gen_85, 'Combined_85', fid, count)
        russia = predict_model(model_5, models_vi_gen_old, 'Russia', fid, count)
        russia_85 = predict_model(model_6, models_vi_gen_85, 'Russia_85', fid, count)

        final.append(mtbs_nbac)
        final.append(mtbs_nbac_85)
        final.append(combined)
        final.append(combined_85)
        final.append(russia)
        final.append(russia_85)

        
print("Done Processing")

final = pd.concat(final).reset_index(drop=True)

final['FID'] = final['FID'].astype(int)

final.to_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/grid_metrics.csv", index = False)



/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipy

Done Processing


In [50]:
't'

't'

fid 206 has different fires (chunked) for the old and 85 method.  WHy??? the old method haS slightly smaller buffers for the fires I think.  I need to fix that. 

In [44]:
fid


206

Now save individual shapefiles which have the differences from russia model which is our main comparison

In [64]:

from pyproj import CRS

#merge back to the original shapefile for plotting
grid_metrics = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/grid_metrics.csv")

#fishnet good
# fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/grid.shp')
fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ea_grid_clip/ea_grid_clip.shp')
# Defining the Albers Equal Area projection parameters
# albers_ea_projection = CRS("+proj=aea +lat_0=56 +lon_0=100 +lat_1=50 +lat_2=70 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")

#Transforming the GeoDataFrame to the new projection
#fish_good= fish_good.to_crs(albers_ea_projection)


fish_good = fish_good.rename(columns = {'Id': 'FID'})


# # 
# fish_good['FID'] = fish_good['FID'].astype(int)

merged = fish_good.merge(grid_metrics, on = 'FID', how = 'inner')

merged = merged[['FID', 'Model', 'Count', 'IOU', 'Precision', 'Recall', 'F-1', 'Accuracy', 'geometry']]

#round floats to 2 digits
merged.loc[:, merged.select_dtypes(include=['float64']).columns] = merged.select_dtypes(include=['float64']).round(2)

#create a new column which will difference the combined model and the north america model from eurasia, do it so we subtract from russia, so larger values are better for russia
na = merged[merged['Model'] == 'MTBS_NBAC']
na_85 = merged[merged['Model'] == 'MTBS_NBAC_85']
combined = merged[merged['Model'] == 'Combined']
combined_85 = merged[merged['Model'] == 'Combined_85']
russ = merged[merged['Model'] == 'Russia']
russ_85 = merged[merged['Model'] == 'Russia_85']

na_russ_diff = russ['IOU'].values - na['IOU'].values
na_russ_85_diff = russ_85['IOU'].values - na_85['IOU'].values
combined_russ_diff = russ['IOU'].values - combined['IOU'].values
combined_russ_85_diff = russ_85['IOU'].values - combined_85['IOU'].values

#for each unique model loop through and save individual files, this is for making maps in arc later easier
models = merged['Model'].unique()

for m in models:
    
    sub = merged[merged['Model'] == m]
    sub['na_russ_IOU_diff'] = na_russ_diff
    sub['na_russ_85_IOU_diff'] = na_russ_85_diff
    sub['combined_russ_IOU_diff'] = combined_russ_diff
    sub['combined_russ_85_IOU_diff'] = combined_russ_85_diff
   
    
    sub.to_file(os.path.join("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial", f"grid_metrics_{m}.shp"))

# merged.head()


/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be s

In [66]:
sub

,FID,Model,Count,IOU,Precision,Recall,F-1,Accuracy,geometry,na_russ_IOU_diff,na_russ_85_IOU_diff,combined_russ_IOU_diff,combined_russ_85_IOU_diff
5,0,Russia_85,36,0.54,0.69,0.69,0.68,0.93,"POLYGON ((100.00000 60.00000, 100.00000 55.000...",-0.01,0.04,-0.11,0.02
11,10,Russia_85,10,0.83,0.90,0.91,0.90,0.97,"MULTIPOLYGON (((161.38889 69.45915, 161.37877 ...",-0.06,0.02,-0.10,0.03
17,11,Russia_85,21,0.62,0.73,0.82,0.76,0.87,"MULTIPOLYGON (((130.00000 50.26354, 130.00000 ...",-0.08,0.00,-0.09,-0.06
23,13,Russia_85,3,0.79,0.94,0.83,0.87,0.97,"POLYGON ((160.00000 69.75901, 160.00000 65.000...",-0.01,0.05,-0.04,0.10
29,15,Russia_85,15,0.50,0.61,0.81,0.64,0.76,"POLYGON ((140.00000 60.00000, 135.00000 60.000...",-0.08,-0.13,-0.15,-0.17
35,18,Russia_85,7,0.80,0.95,0.83,0.89,0.96,"POLYGON ((105.00000 65.00000, 100.00000 65.000...",-0.05,-0.06,-0.06,-0.02
41,20,Russia_85,17,0.51,0.65,0.71,0.61,0.94,"MULTIPOLYGON (((108.91245 54.93700, 108.91673 ...",0.01,-0.01,-0.03,-0.04
47,26,Russia_85,2,0.72,0.99,0.73,0.84,0.97,"MULTIPOLYGON (((99.67921 54.83453, 99.70813 54...",-0.01,-0.01,-0.02,-0.02
53,31,Russia_85,14,0.80,0.84,0.93,0.89,0.94,"MULTIPOLYGON (((137.06602 55.00000, 136.91397 ...",0.05,-0.02,0.02,-0.02
59,40,Russia_85,1,0.27,0.27,1.00,0.42,0.53,"POLYGON ((154.29749 59.63750, 154.25888 59.601...",0.19,-0.20,0.03,-0.61


Now do ecoregions

In [68]:
#first these are all the good anna polygon ids, use this for joining later
# good_ids = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/good_polys_anna.shp')

GPUS = ["GPU:0", "GPU:1", "GPU:2", "GPU:3"]
strategy = tensorflow.distribute.MirroredStrategy() #can add GPUS here to select specific ones
print('Number of devices: %d' % strategy.num_replicas_in_sync) 

#for the grids I have two ids, FID which is the fishnet grid cells to loop through, and ID which is teh good anna polygon nids
fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/ecoregions.shp')

#all the fishnet ids to loop through
all_fid = fish_good['Grid_ID'].unique().tolist()

#get all the testing full pathways to predict on, will need to filter fish good with this
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv')['ID'].tolist()


#now I need to get the chunked files which match the fire ids to make new training, validation and testing times
#path to the chunked files
chunked_85 =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128')
chunked_old =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128')

def filter_chunked_85(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

def filter_chunked2(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    # filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

def filter_chunked_old(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked


#new dnbr composite method
testing_names_85 = filter_chunked_85(testing_names, chunked_85)
testing_names_old = filter_chunked_old(testing_names, chunked_old)



#save all dataframes
final = []

for fid in all_fid:
    
    # print(f"Processing {fid}")
    
    #sub shapefile for the grid
    sub_grid = fish_good[fish_good['Grid_ID'] == fid]
                         
    #get the anna fire ids in this fid
    anna_in_fid = sub_grid['Fire_ID'].unique().tolist()
                        
    count = len(anna_in_fid)
    
    #get full pathway to the anna ids in the fids for new dnbr method
    model_test_85 = filter_chunked2(anna_in_fid, testing_names_85)
    model_test_old = filter_chunked2(anna_in_fid, testing_names_old)
    
    #get the batch sie
    if len(model_test_85) <= 45:
        
        batch_size_85 = len(model_test_85)
       
    else:
        batch_size_85 = 45
        
    if len(model_test_old) <= 45:
        
        batch_size_old = len(model_test_old)
       
    else:
        batch_size_old = 45

        
    if len(model_test_85) > 0 and (len(model_test_old)) > 0:
    
        #create an image generator for this fid and then predict
        models_vi_gen_85 =  img_gen_vi(batch_size_85, img_size, model_test_85)
        models_vi_gen_old =  img_gen_vi(batch_size_old, img_size, model_test_old)

        mtbs_nbac = predict_model(model_1, models_vi_gen_old, 'MTBS_NBAC', fid, count)
        mtbs_nbac_85 = predict_model(model_2, models_vi_gen_85, 'MTBS_NBAC_85', fid, count)
        combined = predict_model(model_3, models_vi_gen_old, 'Combined', fid, count)
        combined_85 = predict_model(model_4, models_vi_gen_85, 'Combined_85', fid, count)
        russia = predict_model(model_5, models_vi_gen_old, 'Russia', fid, count)
        russia_85 = predict_model(model_6, models_vi_gen_85, 'Russia_85', fid, count)

        final.append(mtbs_nbac)
        final.append(mtbs_nbac_85)
        final.append(combined)
        final.append(combined_85)
        final.append(russia)
        final.append(russia_85)

        
print("Done Processing")

final = pd.concat(final).reset_index(drop=True)

final.to_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/ecoregion_metrics.csv", index = False)




Number of devices: 1


/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_885367/1406970591.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipy

Done Processing


ValueError: invalid literal for int() with base 10: 'Montane Sub-Arctic'

In [74]:
final.to_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/ecoregion_metrics.csv", index = False)

In [69]:
't'

't'

In [75]:
from pyproj import CRS

#merge back to the original shapefile for plotting
grid_metrics = pd.read_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/ecoregion_metrics.csv")

#fishnet good
# fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/grid.shp')
fish_good = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/raw_files/EcoRegion_AlbEAadj/EcoRegion_AlbEAadj/EcoRegion_g.shp')
# Defining the Albers Equal Area projection parameters
# albers_ea_projection = CRS("+proj=aea +lat_0=56 +lon_0=100 +lat_1=50 +lat_2=70 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")

# Transforming the GeoDataFrame to the new projection
# fish_good= fish_good.to_crs(albers_ea_projection)

fish_good = fish_good.rename(columns = {'ecoregion': 'FID'})


# # 
# fish_good['FID'] = fish_good['FID'].astype(int)

merged = fish_good.merge(grid_metrics, on = 'FID', how = 'inner')

merged = merged[['FID', 'Model', 'Count', 'IOU', 'Precision', 'Recall', 'F-1', 'Accuracy', 'geometry']]

#round floats to 2 digits
merged.loc[:, merged.select_dtypes(include=['float64']).columns] = merged.select_dtypes(include=['float64']).round(2)

##create a new column which will difference the combined model and the north america model from eurasia, do it so we subtract from russia, so larger values are better for russia
na = merged[merged['Model'] == 'MTBS_NBAC']
na_85 = merged[merged['Model'] == 'MTBS_NBAC_85']
combined = merged[merged['Model'] == 'Combined']
combined_85 = merged[merged['Model'] == 'Combined_85']
russ = merged[merged['Model'] == 'Russia']
russ_85 = merged[merged['Model'] == 'Russia_85']

na_russ_diff = russ['IOU'].values - na['IOU'].values
na_russ_85_diff = russ_85['IOU'].values - na_85['IOU'].values
combined_russ_diff = russ['IOU'].values - combined['IOU'].values
combined_russ_85_diff = russ_85['IOU'].values - combined_85['IOU'].values

#for each unique model loop through and save individual files, this is for making maps in arc later easier
models = merged['Model'].unique()

for m in models:
    
    sub = merged[merged['Model'] == m]
    sub['na_russ_IOU_diff'] = na_russ_diff
    sub['na_russ_85_IOU_diff'] = na_russ_85_diff
    sub['combined_russ_IOU_diff'] = combined_russ_diff
    sub['combined_russ_85_IOU_diff'] = combined_russ_85_diff
    
    sub.to_file(os.path.join("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial", f"ecoregion_metrics_{m}.shp"))


# merged.to_file("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/ecoregion_metrics.shp")

# merged.head()

/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/spotter5/.conda/envs/deeplearning3/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
't'

Get Test IOU across all testing fires for the old dNBR method

In [4]:
def predict_model_single(model, generator, name):
    
    '''
    model: tensorflow model to predict
    generator: keras generator with the images to predict on
    name: string, model name\
    fid: variable I was looping through
    count: count retained earlier
    '''
    #get the results from the nbac and mtbs model
    model_1_res = model.evaluate_generator(generator, 100)

    iou = np.round(model_1_res[-2], 2)
    precision = np.round(model_1_res[-5], 2)
    recall = np.round(model_1_res[-4], 2)
    f1 = np.round(model_1_res[-3], 2)
    accuracy = np.round(model_1_res[-1], 2)

    #make new dataframe with scores
    in_df = pd.DataFrame({
        'Model': [name],
        'IOU': [iou],
        'Precision': [precision],
        'Recall': [recall],
        'F-1': [f1],
        'Accuracy': [accuracy]
                        }, index=[0])  # Explicitly setting index to [0] for a single row

    return in_df


training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_training_files_full_fire.csv')['ID'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_validation_files_full_fire.csv')['ID'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv')['ID'].tolist()

chunked =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_mtbs_subs_0_16')

def filter_chunked(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/nbac_training/modis_mtbs_subs_0_16/' + i for i in filtered_chunked]
    return filtered_chunked

training_names = filter_chunked(training_names, chunked)
validation_names = filter_chunked(validation_names, chunked)
testing_names = filter_chunked(testing_names, chunked)



#image generator
batch_size = 45
img_size = (16, 16)
models_vi_gen =  img_gen_vi(batch_size, img_size, testing_names)

# russia = predict_model_single(model_5, models_vi_gen, 'Russia')
mtbs_nbac = predict_model_single(model_1, models_vi_gen, 'MTBS_NBAC')
# combined = predict_model_single(model_3, models_vi_gen, 'Combined')


# final = pd.concat([russia, mtbs_nbac, combined])

# final.to_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/80_10_10_comparisons_old_dnbr.csv", index = False)
        

/tmp/ipykernel_2152056/1058569666.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)


In [5]:
mtbs_nbac

,Model,IOU,Precision,Recall,F-1,Accuracy
0,MTBS_NBAC,0.84,0.95,0.87,0.91,0.99


Now do it for the 85 dNBR method

In [7]:
def predict_model_single(model, generator, name):
    
    '''
    model: tensorflow model to predict
    generator: keras generator with the images to predict on
    name: string, model name\
    fid: variable I was looping through
    count: count retained earlier
    '''
    #get the results from the nbac and mtbs model
    model_1_res = model.evaluate_generator(generator, 100)

    iou = np.round(model_1_res[-2], 2)
    precision = np.round(model_1_res[-5], 2)
    recall = np.round(model_1_res[-4], 2)
    f1 = np.round(model_1_res[-3], 2)
    accuracy = np.round(model_1_res[-1], 2)

    #make new dataframe with scores
    in_df = pd.DataFrame({
        'Model': [name],
        'IOU': [iou],
        'Precision': [precision],
        'Recall': [recall],
        'F-1': [f1],
        'Accuracy': [accuracy]
                        }, index=[0])  # Explicitly setting index to [0] for a single row

    return in_df


training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_training_files_full_fire.csv')['ID'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_validation_files_full_fire.csv')['ID'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv')['ID'].tolist()

chunked =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128')

def filter_chunked(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

training_names = filter_chunked(training_names, chunked)
validation_names = filter_chunked(validation_names, chunked)
testing_names = filter_chunked(testing_names, chunked)



#image generator
batch_size = 45
img_size = (128, 128)
models_vi_gen =  img_gen_vi(batch_size, img_size, testing_names)

# russia = predict_model_single(model_5, models_vi_gen, 'Russia')
russia_85 = predict_model_single(model_6, models_vi_gen, 'Russia_85')

# mtbs_nbac = predict_model_single(model_1, models_vi_gen, 'MTBS_NBAC')
mtbs_nbac_85 = predict_model_single(model_2, models_vi_gen, 'MTBS_NBAC_85')
# combined = predict_model_single(model_3, models_vi_gen, 'Combined')
combined_85 = predict_model_single(model_4, models_vi_gen, 'Combined_85')

final2 = pd.concat([russia_85,  mtbs_nbac_85, combined_85])

for_ex = pd.concat([final, final2])
for_ex.to_csv("/explore/nobackup/people/spotter5/cnn_mapping/Russia/model_iou_spatial/80_10_10_comparisons.csv", index = False)
        

/tmp/ipykernel_878731/1522778963.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_878731/1522778963.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)
/tmp/ipykernel_878731/1522778963.py:11: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  model_1_res = model.evaluate_generator(generator, 100)


In [8]:
for_ex


,Model,IOU,Precision,Recall,F-1,Accuracy
0,Russia,0.714404,0.938702,0.750332,0.815288,0.933501
0,MTBS_NBAC,0.720209,0.924550,0.766354,0.817878,0.937555
0,Combined,0.765555,0.916036,0.825903,0.852501,0.947957
0,Russia_85,0.734176,0.844196,0.832408,0.824704,0.939955
0,MTBS_NBAC_85,0.684287,0.934820,0.717266,0.789611,0.931909
0,Combined_85,0.678271,0.945588,0.704758,0.787747,0.929774
